In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 16 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

06 February 2023 ; 16:34:50


# Features, model

In [3]:
features = ['VORP','WS','OWS','BPM','PER','PTS','FT','FTA','OBPM','FG','USG%','Seed','MP']#,'GS'] # GS !!!
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/mvp/mvp_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') 

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,24,0,9.1,1.5,3.5,0.440,...,2.42,113.98,111.67,2.32,2.23,113.97,111.82,2.16,1,1
1,Bobby Portis,PF,27,MIL,47,14,26.8,6.0,12.0,0.499,...,2.42,113.98,111.67,2.32,2.23,113.97,111.82,2.16,0,1
2,Brook Lopez,C,34,MIL,52,52,30.3,5.4,10.9,0.496,...,2.42,113.98,111.67,2.32,2.23,113.97,111.82,2.16,0,0
3,George Hill,PG,36,MIL,35,0,19.1,1.7,3.8,0.447,...,2.42,113.98,111.67,2.32,2.23,113.97,111.82,2.16,0,1
4,Giannis Antetokounmpo,PF,28,MIL,42,42,33.6,11.5,21.3,0.539,...,2.42,113.98,111.67,2.32,2.23,113.97,111.82,2.16,0,0


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] >= season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player' ,'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'Share']]
res.head()

,Player,Tm,PER,VORP,WS,PTS,Seed,Share
0,Nikola Jokić,DEN,32.0,10.517391,18.360870,24.8,1,0.864046
1,Luka Dončić,DAL,30.5,9.246809,14.480851,33.4,6,0.579571
2,Joel Embiid,PHI,30.7,7.380000,14.350000,33.4,3,0.525588
3,Jayson Tatum,BOS,24.3,5.904000,12.136000,30.9,1,0.525521
4,Giannis Antetokounmpo,MIL,28.5,6.833333,11.128571,32.4,2,0.493868


In [7]:
res.to_csv(path + f'Results/mvp/results_week_{week}.csv', index = None)